In [ ]:
import azureml.core
from azureml.core import Experiment, Environment, Workspace, Dataset, Datastore, ScriptRunConfig
from azureml.core.conda_dependencies import CondaDependencies
import os
import shutil

# check core SDK version number

print("Azure ML SDK Version: ", azureml.core.VERSION)

In [ ]:
# load workspace configuration from the config.json file in the current folder.
ws = Workspace.from_config()
# get metadata about the workspace
print(ws.name, ws.location, ws.resource_group, sep='\t')
# list the registered datastores
ws.datastores

In [ ]:
# register our data share containing image data in this workspace
Datastore.register_azure_blob_container(
    account_key = 'jP7J8NN8DuIcDblyLkw2FElcyhtxlHSVZC2j+07ZboUtrLSQGKvwALZDoE9hfuKP9pxY2p3yzLQP84R1nY9ErQ==',
    workspace = ws,
    datastore_name = 'solarDataBlob',
    container_name = 'solar',
    account_name = 'aiprojects')

In [ ]:
# create a file dataset that can be used in training
datastore = Datastore.get(workspace = ws, datastore_name = 'solardatablob')
datastore_paths = [(datastore, 'CPK_solar'), (datastore, 'NC_solar')]
cpk_dataset = Dataset.File.from_files(path = datastore_paths[0])
nc_dataset = Dataset.File.from_files(path = datastore_paths[1])

cpk_dataset = cpk_dataset.register(
    workspace=ws,
    name='gee-cpk-solar-data',
    description='training and eval TFRecords for solar arrays exported from GEE',
    create_new_version=True)

nc_dataset = nc_dataset.register(
    workspace=ws,
    name='gee-nc-solar-data',
    description='training and eval TFRecords for solar arrays exported from GEE',
    create_new_version=True)

In [ ]:
# create an environment for the first time

envs = Environment.list(workspace = ws)
# well start with a pre-built tensorflow environment
env = envs.get('AzureML-TensorFlow-2.3-GPU')
env

# define packages to be installed using CondaDependencies
# get the packages that are already part of the pre-built environment
conda_dep = env.python.conda_dependencies
# list packages to install
pip_packages = ['matplotlib', 'rasterio', 'tensorboard']

# add each package to the existing conda dependencies
for package in pip_packages:
    conda_dep.add_pip_package(package)

# double check all the packages are there
conda_dep.serialize_to_string()
# conda_dep = CondaDependencies.create(
#     pip_packages=pip_packages)

# Now update the conda dependencies of the python environment
env.python.conda_dependencies=conda_dep

# # Register environment to re-use later
env.name = 'solar-training'
env.register(workspace = ws)

In [ ]:
# create an environment for the first time

envs = Environment.list(workspace = ws)
# well start with a pre-built tensorflow environment
env = envs.get('AzureML-tensorflow-2.4-ubuntu18.04-py37-cuda11-gpu')
# env

# define packages to be installed using CondaDependencies
# get the packages that are already part of the pre-built environment
conda_dep = CondaDependencies()
# list packages to install
pip_packages = ['matplotlib', 'rasterio', 'tensorboard']

# add each package to the existing conda dependencies
for package in pip_packages:
    conda_dep.add_pip_package(package)

# double check all the packages are there
conda_dep.serialize_to_string()
# conda_dep = CondaDependencies.create(
#     pip_packages=pip_packages)

# Now update the conda dependencies of the python environment
env.python.conda_dependencies=conda_dep

# # Register environment to re-use later
env.name = 'landcover-training'
env.register(workspace = ws)